In [49]:
import string

In [ ]:
# onp functions

In [7]:
def getVariables(expression):
    variables = [letter for letter in expression if letter.isalpha()]
    return ''.join(sorted(set(variables)))


In [8]:
def validate(expression):
    variables = string.ascii_lowercase + 'T' + 'F'
    operators = "&>|/>"
    state = True
    counter = 0

    for char in expression:
        if char == '(': counter += 1
        if char == ')': counter -= 1
        if counter < 0: return False
        if state:
            if char == '~': state = True
            elif char in variables: state = False
            elif char in ')' + operators: return False
        else:
            if char in operators: state = True
            elif char in '(' + variables + '~': return False
    if counter != 0: return False
    return not state

In [9]:
def divideExpression(expression, operators):
    counter = 0
    for i in range(len(expression) - 1, -1, -1):       # zmieniłem 0 na -1
        if expression[i] == ')': counter += 1
        if expression[i] == '(': counter -= 1
        if expression[i] in operators and counter == 0: return i
    return -1

In [40]:
def onp(expression):
    operators = "&>|/>"
    while expression[0] == '(' and expression[-1] == ')' and validate(expression):
        expression = expression[1:-1]

    for p in [result for result in [divideExpression(expression, operator) for operator in ">|&/"] if result >= 0] :
        return onp(expression[:p]) + onp(expression[p + 1:]) + expression[p]
        
    p = divideExpression(expression, '~')
    if p >= 0:
        return onp(expression[p + 1:]) + expression[p]
        
    return expression

In [16]:
def substitude(expression, variables, values):
    mappedExpr = list(expression)
    for i in range(len(expression)):
        pos = variables.find(expression[i])
        if pos >= 0: mappedExpr[i] = values[pos]
    return ''.join(mappedExpr)

In [43]:
def evaluate(expression, values):
    variables = getVariables(expression)
    print(variables)
    mappedExpr = onp(substitude(expression, variables, values))
    print(mappedExpr)
    stack = list("")
    for x in mappedExpr:
        if x in '01': stack.append(int(x))
        else:
            y1 = not not stack.pop()
            if x == '~': 
                stack.append(not y1)
            else: 
                y2 = not not stack.pop()
                if x == '&': stack.append(y1 and y2)
                elif x == '|': stack.append(y1 or y2)
                elif x == '>': stack.append(y1 or (not y2))
    return stack.pop()

In [46]:
x = "a&~c|a&b"
print(x)
print(validate(x))
evaluate(x, "000")

a&~c|a&b
True
abc
00~&00&|


False

In [ ]:
def binaryGenerator(n):
    if n == 0: yield ""
    else:
       for c in binaryGenerator(n-1):
            yield "0" + c
            yield "1" + c

In [47]:
# Quine Mccluskey algorithm

In [50]:
def latch(vec1, vec2):
    diff = 0
    result = ""
    for c1, c2 in zip(vec1, vec2):
        if c1 == c2:
            result += c1
        else:
            diff += 1
            result += "-"
    if diff == 1: return result
    else: return False

In [51]:
def readData():
    data = open("data.txt", "r")
    result = set(data.read().splitlines())
    data.close()
    return result

In [52]:
def reduce(S):
    b2 = False
    result = set()
    for elem1 in S:
        b1 = False
        for elem2 in S:
            vec = latch(elem1, elem2)
            if vec:
                result.add(vec)
                b1 = b2 = True
        if not b1: result.add(elem1)
    if b2: return reduce(result)
    return result

In [53]:
def minimalize(expression):
    mainResult = ""
    for vec in expression:
        result = ""
        for elem in range(len(vec)):
            if vec[elem] == "-": continue
            if vec[elem] == "0": result += "~"
            result += "abcdefghij"[elem]+"&"
        mainResult += "(" + result[:-1] + ") | "
    return mainResult[:-2]
